In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium
from urllib.parse import  urlparse
import time
import lxml
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import re
import pickle
from contextlib import suppress

In [ ]:
# 네이버 카페 게시판의 게시물을 수집하는 크롤러(제목, 본문, 등록일

In [49]:
# 기본 정보

cafe_code = "anycallusershow" #str
board_url = "https://cafe.naver.com/ArticleList.nhn?search.clubid=13764661&search.menuid=1034&search.boardtype=L&search.totalCount=151&search.cafeId=13764661" # 게시판 url
page_range = 1  #int
post_num = []
contents_list = []

In [50]:
# 대상 게시판의 지정된 페이지만큼 글 번호만 추출 (리스트화)

driver = None
driver = webdriver.Chrome("chromedriver")

for i in range(page_range):
    url = board_url + "&search.page=" + str(i+1)  # 주소 지정
        
    driver.get(url)
    time.sleep(1) # implicity_wait, 암묵적, 파싱 대기시간. / #time.sleep은 무조건 대기
    driver.switch_to.frame('cafe_main')
    
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.find_all('div', {'class': 'board-number'})
    
    for n in a:
        post_num.append(re.search("[0-9]{2,8}", str(n)).group())
    
driver.quit()


In [51]:
print(post_num) # 내용 확인


['3760682', '3760358', '3760303', '3760287', '3760144', '3760107', '3759894', '3759543', '3759082', '3758796', '3758544', '3758136', '3758116', '3758102', '3757958']


In [55]:
# 대상 게시물의 제목, 본문, 날짜 수집

driver = None
driver = webdriver.Chrome("chromedriver")

temp_title = []
temp_content = []
temp_date = []
    
for i in post_num:
    url = "https://cafe.naver.com/" + cafe_code + "/" + i
    
    driver.get(url)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    content = soup.find('div', class_='ArticleContentBox')    

    with suppress(AttributeError):   # 제목 없는 게시글
        temp_title.append(content.find('h3',class_='title_text').text.strip()) 
        temp_content.append(content.find("div", class_="article_viewer").text.strip())
        temp_date.append(content.find('div',class_='article_info').find('span',class_='date').text.strip())
        
driver.quit()        

In [56]:
# 내용 확인
arr = np.vstack([np.array(temp_title), np.array(temp_content), np.array(temp_date)])
df = pd.DataFrame(arr).T
df

                                0  \
0                     S21울트라 기변~~   
1                s21u 8월 업뎃 후기 인증   
2                 전 뽑기? 성공했네요 ㅋㅋㅋ   
3   S21_엑시노스2100은 명백한 결함이고 리콜감입니다   
4                            생존신고   
5            속도 강화 전 후가 이렇게 차이나네요   
6             21+ 업데이트 후 카메라 실행속도   
7               S21은 울트라가 제일 낫네요.   
8                        업뎃 전후 벤치   
9             이번 업데이트 후 발열 어떠신가요?   
10           + 팬텀실버 산지 3일차 허접한 후기   
11          가족폰인데 업뎃후 에어컨 벤치결과입니다   
12                   업뎃후에 더떨어졌는데;   
13              갤럭시S21 울트라 긱벤치 점수   
14                        업뎃후 긱벤치   

                                                    1                  2  
0   질문/추천 금지 X​s21울트라로 기변했습니다.침수된 s20+는 as비용만 80~9...  2021.08.10. 07:09  
1   질문/추천 금지 X​안녕하세요?​4월 말에 구매해서 지금까지 울트라 잘 사용하고 있...  2021.08.09. 17:50  
2   ​사용기종: s21​최대가 34~35도고 카메라 사용시 36도까지 찍었네요 ㅋㅋㅋㅋ...  2021.08.09. 17:00  
3   성능을 반으로 줄여서 발열을 잡으면 그게 플래그쉽입니까? \n\n\n\n\n\n\n...  2021.08.09. 16:46  
4   놋텐플에서 기변으로 돌아왔습니다 바꾼지5일째 지만 

In [ ]:
# temp_dict['content'] = content.find("div", class_="article_viewer").text.strip()

#         temp_dict['date'] = content.find('div',class_='article_info').find('span',class_='date').text.strip()
#         contents_list.append(temp_dict)